[boto docs](http://boto3.readthedocs.io/en/latest/reference/services/glue.html)

In [ ]:
import os
print(os.environ['PATH'])
print(os.environ['JAVA_HOME'])
print(os.environ['CLASSPATH'])

This notebook attempts to create a database in glue and a crawler that interatively updates the database with new data from an S3 bucket

In [ ]:
# Get the packages I need
import pandas as pd
import numpy as np
# import boto3
from io import StringIO
from pyathenajdbc import connect
import data_engineering_utils.glue as glue_utils
import data_engineering_utils.meta as meta_utils
import data_engineering_utils.basic as basic_utils
import copy
import datetime as dt

staging_bucket = 'alpha-karik-glue-tests'
test_data_folder = '/Users/karik/Documents/projects/test_data/'

Get some data

In [ ]:
# Prep data
# For each dataset we need to create a table.meta.json file for each table
# In this example I just set everything to default (all cols are strings with a default description)
name_list = ['department', 'employees', 'sales']
for name in name_list :
    d = pd.read_csv(test_data_folder + name + '.csv')
    cols = meta_utils.get_col_types_from_df(d)
    table_meta = meta_utils.Table_metadata(table_name = name,
                                           bucket = staging_bucket,
                                           columns = cols,
                                           table_description = name + ' table description')
    basic_utils.write_json(table_meta, test_data_folder + 'meta/' + name + '.meta.json')

In [ ]:
# As an example can also do this (we'll improve the meta data for sales):
sales_cols = basic_utils.get_csv_header(file_path = (test_data_folder + 'sales.csv'),
                            convert_to_lower = True)

cols = meta_utils.create_columns_meta(column_names = sales_cols,
                                      data_type_overrides = {'qtr' : 'bigint',
                                                            'sales' : 'double'},
                                      description_overrides = {'staff_id' : 'id of sales staff',
                                                               'qtr' : 'quarter at which sales were made',
                                                               'sales' : 'total sales for that quarter'})
table_meta = meta_utils.Table_metadata(table_name = 'sales',
                                       bucket = staging_bucket,
                                       columns = cols,
                                       table_description = 'Table containing sales totals for each quarter')
basic_utils.write_json(table_meta, test_data_folder + 'meta/sales.meta.json')

In [ ]:
# Now we have the meta data and csvs for each table we can send them to S3
# This should work but doesn't cos of permissions
db_name = 'my_first_database'
glue_utils.create_database(db_description= 'Some test database', db_name = db_name)

# Read in test data and pump it into S3 buckets and then add to my db
name_list = ['department', 'employees', 'sales']
for name in name_list :
    # Read data and meta
    d = pd.read_csv(test_data_folder + name + '.csv')
    m = basic_utils.read_json(test_data_folder + 'meta/' + name + '.meta.json')
    
    # Get the table spec for glue
    table_spec = glue_utils.get_glue_table_spec_from_meta(meta_object = m, template_type = 'csv')

    # Save df in S3 bucket
    path = staging_bucket + '/' + name + '/' + dt.datetime.today().strftime("%Y_%m_%d") + '.csv'
    glue_utils.df_to_csv_s3(df = d, bucket = staging_bucket, path = path)
    
    # Create glue table
    glue_utils.create_table_from_def(db_name = db_name, table_name = name, table_spec = table_spec)

In [ ]:
s3_path_to_athena_out = "s3://" + staging_bucket + '/athena_outputs'
conn = connect(s3_staging_dir=s3_path_to_athena_out,
               region_name='eu-west-1')
sql = """
select * from alpha-karik-glue-tests.department
"""
pd.read_sql(sql, conn)